> TESTING TESTING TESTING TESTING 

## Algo to start a speculative Optiontrade with Riskmanagment and Profit-taking using a OCA-order


A call option is choosen to jump on an expected price-move of a stock.
The order is submitted as OCA-Bracket-Order

Please be aware, that this algo is not implementing a profitable trading strategy. You will loose money if you use it in production.


> TESTING TESTING TESTING TESTING 
>
> This Example is under construction

First, load ib-ruby environment

In [ ]:
require_relative '../../lib/iruby_init.rb'
require 'rover-df'

Ensure that a Demo-Account is used

In [ ]:
# raises an error and stops the script unless a paper trading account is used
exit unless G.clients.first.test_environment?

# initialize a proper client regardles of the type of account used
the_client =  G.clients.last; nil

-- 

Apple is the underlying. 

-- 

In [ ]:
aapl =  Stock.new( symbol: :aapl ).verify.first
puts aapl.as_table
puts "Market-price: #{ref_price = aapl.market_price}"
# in case that no market-price is transmitted ( happens occasionally if the code is run on a paper account )
ref_price ||=  150; nil   # suppress output

Get out of the money options,  do not try ask for a market-price again. Use the ref price instead.

In [ ]:
apple_options =  aapl.otm_options right: :call, ref_price: ref_price
puts "*"*20
puts "OTM Strikes"
puts apple_options.keys.join(" . ")

Using the second (regular) expiry of the third otm-strike

In [ ]:
the_strike = apple_options.keys.at 2

#####  This is the subset of the Option-Chain to select a candidate to trade
aapl_base =  apple_options[the_strike]
puts aapl_base.as_table



Now, select the appropiate option.
  
For simplicity, this demo just picks the third option from the list

In [ ]:
apple_option =  aapl_base.at 2
puts apple_option.as_table
apple_option.request_greeks
puts "Ref-price of the Option:  #{apple_option.greek.option_price.round(4)}"

#### Setup the Trade

* The Entry will be  Ref-price - 3 %

* The Stop-Price is -50 % of the reference price

* Profit-Taking happens on + 30 % from the reference price

In [ ]:
adjusted_price = -> (rel) do 
   rel =  rel.to_f / 100 if rel.abs > 1
  # adjust round to a proper value if the order is not accepted due to improper price increment
   (apple_option.greek.option_price + (apple_option.greek.option_price.to_f * rel )).round(1)
end

Place the prepared orders

In [ ]:
C.send_message :RequestGlobalCancel
G.update_orders
the_client =  G.clients.last
#exit unless G.clients.orders.flatten.empty?

puts "market_price: #{apple_option.market_price}"

puts "removed pending orders"
puts "\nestablished orders:"
buy_order = IB::Limit.order :size => 10,
                            :price => adjusted_price[ -3 ],
                            :action => :buy,
                            :algo_strategy => '',
                            :transmit => true

the_client.place contract: apple_option, order: buy_order
puts buy_order.to_human

In [ ]:
stop_order = IB::SimpleStop.order :size => 10,
                            :price => adjusted_price[ -50 ],
                            :action => :sell,
                            :oca_group => 'A'

the_client.place contract: apple_option, order: stop_order

puts stop_order.to_human

In [ ]:
profit_order = IB::Limit.order  :size => 10,
                          :price => adjusted_price[ 30 ] ,
                          :action => :sell,
                          :oca_group => 'A'
                          

the_client.place contract: apple_option, order: profit_order

puts profit_order.to_human

In [ ]:
oca_order = IB::Order.new order_type: :one_cancels_all,
                          oca_group: 'A',
                          oca_type: 1,
                          account: the_client.account


C.place_order oca_order, apple_option
                          

               
puts oca_order.to_human

Request all active orders, update the bookkeeping on the clients and print a summary

In [ ]:
G.update_orders
puts the_client.orders.to_human

In [ ]:
#C.send_message :RequestGlobalCancel